# TP 1: LDA/QDA y optimización matemática de modelos

# Intro teórica

## Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

## Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

## LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

## Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

## Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

# Código provisto

Con el fin de no retrasar al alumno con cuestiones estructurales y/o secundarias al tema que se pretende tratar, se provee una base de código que **no es obligatoria de usar** pero se asume que resulta resulta beneficiosa.

In [1]:
import numpy as np
import pandas as pd
import numpy.linalg as LA
from scipy.linalg import cholesky, solve_triangular
from scipy.linalg.lapack import dtrtri

## Base code

In [2]:
class BaseBayesianClassifier:
  def __init__(self):
    pass

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    # Q3: para que sirve bincount?
    return np.log(a_priori)

  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: por que el _fit_params va al final? no se puede mover a, por ejemplo, antes de la priori?

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=int)

    for i in range(m_obs):
      y_hat[i] = self._predict_one(X[:,i].reshape(-1,1))

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [3]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [LA.inv(np.cov(X[:,y.flatten()==idx], bias=True))
                      for idx in range(len(self.log_a_priori))]
    # Q5: por que hace falta el flatten y no se puede directamente X[:,y==idx]?
    # Q6: por que se usa bias=True en vez del default bias=False?
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]
    # Q7: que hace axis=1? por que no axis=0?

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(LA.det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [4]:
class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_covs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0,2,1) @ self.tensor_inv_cov @ unbiased_x

        return 0.5*np.log(LA.det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

In [5]:
class QDA_Chol1(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        LA.inv(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True))
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

In [6]:
class QDA_Chol2(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.Ls = [
        cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True)
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L = self.Ls[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = solve_triangular(L, unbiased_x, lower=True)

    return -np.log(L.diagonal().prod()) -0.5 * (y**2).sum()

In [7]:
class QDA_Chol3(BaseBayesianClassifier):
  def _fit_params(self, X, y):
    self.L_invs = [
        dtrtri(cholesky(np.cov(X[:,y.flatten()==idx], bias=True), lower=True), lower=1)[0]
        for idx in range(len(self.log_a_priori))
    ]

    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True)
                  for idx in range(len(self.log_a_priori))]

  def _predict_log_conditional(self, x, class_idx):
    L_inv = self.L_invs[class_idx]
    unbiased_x =  x - self.means[class_idx]

    y = L_inv @ unbiased_x

    return np.log(L_inv.diagonal().prod()) -0.5 * (y**2).sum()

## Datasets

In [8]:
from sklearn.datasets import load_iris, fetch_openml, load_wine
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

def get_penguins_dataset():
    # get data
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')

    # drop non-numeric columns
    df.drop(columns=["island","sex"], inplace=True)

    # drop rows with missing values
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]

    return df.values, tgt.to_numpy().reshape(-1,1)

def get_wine_dataset():
    # get data
    data = load_wine()
    X_full = data.data
    y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
    return X_full, y_full

def get_letters_dataset():
    # get data
    letter = fetch_openml('letter', version=1, as_frame=False)
    return letter.data, letter.target.reshape(-1,1)

def label_encode(y_full):
    return LabelEncoder().fit_transform(y_full.flatten()).reshape(y_full.shape)

def split_transpose(X, y, test_size, random_state):
    # X_train, X_test, y_train, y_test but all transposed
    return [elem.T for elem in train_test_split(X, y, test_size=test_size, random_state=random_state)]

## Benchmarking

Nota: esta clase fue creada bastante rápido y no pretende ser una plataforma súper confiable sobre la que basarse, sino más bien una herramienta simple con la que poder medir varios runs y agregar la información.

En forma rápida, `warmup` es la cantidad de runs para warmup, `mem_runs` es la cantidad de runs en las que se mide el pico de uso de RAM y `n_runs` es la cantidad de runs en las que se miden tiempos.

La razón por la que se separan es que medir memoria hace ~2.5x más lento cada run, pero al mismo tiempo se estabiliza mucho más rápido.

**Importante:** tener en cuenta que los modelos que predicen en batch (usan `predict` directamente) deberían consumir, como mínimo, $n$ veces la memoria de los que predicen por observación.

In [9]:
import time
from tqdm import tqdm
from numpy.random import RandomState
import tracemalloc

RNG_SEED = 6553

class Benchmark:
    def __init__(self, X, y, n_runs=1000, warmup=100, mem_runs=100, test_sz=0.3, rng_seed=RNG_SEED, same_splits=True):
        self.X = X
        self.y = y
        self.n = n_runs
        self.warmup = warmup
        self.mem_runs = mem_runs
        self.test_sz = test_sz
        self.det = same_splits
        if self.det:
            self.rng_seed = rng_seed
        else:
            self.rng = RandomState(rng_seed)

        self.data = dict()

        print("Benching params:")
        print("Total runs:",self.warmup+self.mem_runs+self.n)
        print("Warmup runs:",self.warmup)
        print("Peak Memory usage runs:", self.mem_runs)
        print("Running time runs:", self.n)
        approx_test_sz = int(self.y.size * self.test_sz)
        print("Train size rows (approx):",self.y.size - approx_test_sz)
        print("Test size rows (approx):",approx_test_sz)
        print("Test size fraction:",self.test_sz)

    def bench(self, model_class, **kwargs):
        name = model_class.__name__
        time_data = np.empty((self.n, 3), dtype=float)  # train_time, test_time, accuracy
        mem_data = np.empty((self.mem_runs, 2), dtype=float)  # train_peak_mem, test_peak_mem
        rng = RandomState(self.rng_seed) if self.det else self.rng


        for i in range(self.warmup):
            # Instantiate model with error check for unsupported parameters
            model = model_class(**kwargs)

            # Generate current train-test split
            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )
            # Run training and prediction (timing or memory measurement not recorded)
            model.fit(X_train, y_train)
            model.predict(X_test)

        for i in tqdm(range(self.mem_runs), total=self.mem_runs, desc=f"{name} (MEM)"):

            model = model_class(**kwargs)

            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )

            tracemalloc.start()

            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()

            _, train_peak = tracemalloc.get_traced_memory()
            tracemalloc.reset_peak()

            model.predict(X_test)
            t3 = time.perf_counter()
            _, test_peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            mem_data[i,] = (
                train_peak / (1024 * 1024),
                test_peak / (1024 * 1024)
            )

        for i in tqdm(range(self.n), total=self.n, desc=f"{name} (TIME)"):
            model = model_class(**kwargs)

            X_train, X_test, y_train, y_test = split_transpose(
                self.X, self.y,
                test_size=self.test_sz,
                random_state=rng
            )

            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()
            preds = model.predict(X_test)
            t3 = time.perf_counter()

            time_data[i,] = (
                (t2 - t1) * 1000,
                (t3 - t2) * 1000,
                (y_test.flatten() == preds.flatten()).mean()
            )

        self.data[name] = (time_data, mem_data)

    def summary(self, baseline=None):
        aux = []
        for name, (time_data, mem_data) in self.data.items():
            result = {
                'model': name,
                'train_mean_ms': time_data[:, 0].mean(),
                'train_std_ms': time_data[:, 0].std(),
                'test_mean_ms': time_data[:, 1].mean(),
                'test_std_ms': time_data[:, 1].std(),
                'mean_accuracy': time_data[:, 2].mean(),
                'train_mem_mean_mb': mem_data[:, 0].mean(),
                'train_mem_std_mb': mem_data[:, 0].std(),
                'test_mem_mean_mb': mem_data[:, 1].mean(),
                'test_mem_std_mb': mem_data[:, 1].std()
            }
            aux.append(result)
        df = pd.DataFrame(aux).set_index('model')

        if baseline is not None and baseline in self.data:
            df['train_speedup'] = df.loc[baseline, 'train_mean_ms'] / df['train_mean_ms']
            df['test_speedup'] = df.loc[baseline, 'test_mean_ms'] / df['test_mean_ms']
            df['train_mem_reduction'] = df.loc[baseline, 'train_mem_mean_mb'] / df['train_mem_mean_mb']
            df['test_mem_reduction'] = df.loc[baseline, 'test_mem_mean_mb'] / df['test_mem_mean_mb']
        return df

## Ejemplo

In [10]:
# levantamos el dataset Wine, que tiene 13 features y 178 observaciones en total
X_full, y_full = get_wine_dataset()

X_full.shape, y_full.shape

((178, 13), (178, 1))

In [11]:
# encodeamos a número las clases
y_full_encoded = label_encode(y_full)

y_full[:5], y_full_encoded[:5]

(array([['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0'],
        ['class_0']], dtype='<U7'),
 array([[0],
        [0],
        [0],
        [0],
        [0]]))

In [12]:
# generamos el benchmark
# observar que son valores muy bajos de runs para que corra rápido ahora
b = Benchmark(
    X_full, y_full_encoded,
    n_runs = 100,
    warmup = 20,
    mem_runs = 20,
    test_sz = 0.3,
    same_splits = False
)

Benching params:
Total runs: 140
Warmup runs: 20
Peak Memory usage runs: 20
Running time runs: 100
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3


In [13]:
# bencheamos un par
to_bench = [QDA]

for model in to_bench:
    b.bench(model)

QDA (TIME): 100%|██████████| 100/100 [00:00<00:00, 589.24it/s]


In [14]:
# como es una clase, podemos seguir bencheando más después
b.bench(TensorizedQDA)

TensorizedQDA (TIME): 100%|██████████| 100/100 [00:00<00:00, 1099.96it/s]


In [15]:
# hacemos un summary
b.summary()

,train_mean_ms,train_std_ms,test_mean_ms,test_std_ms,mean_accuracy,train_mem_mean_mb,train_mem_std_mb,test_mem_mean_mb,test_mem_std_mb
model,,,,,,,,,
QDA,0.153592,0.030723,1.400253,0.057478,0.982407,0.018348,0.000664,0.008037,0.001445
TensorizedQDA,0.155910,0.020562,0.624067,0.033047,0.982593,0.018229,0.000698,0.012093,0.000213


In [16]:
# son muchos datos! nos quedamos con un par nomás
summ = b.summary()

# como es un pandas DataFrame, subseteamos columnas fácil
summ[['train_mean_ms', 'test_mean_ms','mean_accuracy']]

,train_mean_ms,test_mean_ms,mean_accuracy
model,,,
QDA,0.153592,1.400253,0.982407
TensorizedQDA,0.155910,0.624067,0.982593


In [17]:
# podemos setear un baseline para que fabrique columnas de comparación
summ = b.summary(baseline='QDA')

summ

,train_mean_ms,train_std_ms,test_mean_ms,test_std_ms,mean_accuracy,train_mem_mean_mb,train_mem_std_mb,test_mem_mean_mb,test_mem_std_mb,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,,,,,,,
QDA,0.153592,0.030723,1.400253,0.057478,0.982407,0.018348,0.000664,0.008037,0.001445,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,0.155910,0.020562,0.624067,0.033047,0.982593,0.018229,0.000698,0.012093,0.000213,0.985132,2.243754,1.006495,0.664579


In [18]:
summ[[
    'train_mean_ms', 'test_mean_ms','mean_accuracy',
    'train_speedup', 'test_speedup',
    'train_mem_reduction', 'test_mem_reduction'
]]

,train_mean_ms,test_mean_ms,mean_accuracy,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,
QDA,0.153592,1.400253,0.982407,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,0.155910,0.624067,0.982593,0.985132,2.243754,1.006495,0.664579


# Respuestas a preguntas técnicas

**Q3: ¿Para que sirve bincount?**

Sirve para contar la frecuencia de aparición de cada índice o valor del vector original. En este caso se lo divide además por la cantidad total de elementos del vector original con el fin de asignar una probabilidad de aparición para cada evento, siendo esta la función de densidad a priori obtenida a partir de las observaciones hechas.

**Q4: ¿Por qué el `_fit_params` va al final? ¿No se puede mover a, por ejemplo, antes de la priori?**

El método `_fit_params` no puede ir antes del cálculo de la distribución a priori porque el primero depende de la propia función a priori para calcular las matrices necesarias. Ejecutarlo después del momento en el que se utilice también carece de sentido ya que significaría que el modelo debe tratar de predecir valores con parámetros que no han sido entrenados, invalidando los resultados del test que se desee hacer.

**Q5: ¿Por qué hace falta el flatten y no se puede directamente `X[:,y==idx]`?**

La variable `y` es un objeto de tipo `np.Series` de n filas y 1 columna. El filtro que se desea utilizar en los valores del dataframe guardado en la variable `X` solo es aplicable para objetos del tipo lista, de modo que se usa el método `flatten` para transformar a `y` en una lista equivalente y poder filtrar los valores de cada observación correctamente.

**Q6: ¿Por qué se usa `bias=True` en vez del default `bias=False`?**

Porque se desea calcular la expresión con la hipótesis de máxima verosimilitud, que emplea un estimador sesgado que debe usar un factor $ \frac{1}{n}$, lo cual se indica a la función configurando el parámetro `bias=True`.

**Q7: ¿Qué hace `axis=1`? ¿Por qué no `axis=0`?**

Para obtener el valor medio para cada observación del dataset. Hacerlo con el valor `axis=0` indicaría que se promedian los distintos features para una misma observación, lo cual no tiene sentido.

# Consigna QDA

**Notación**: en general notamos

* $k$ la cantidad de clases
* $n$ la cantidad de observaciones
* $p$ la cantidad de features/variables/predictores

**Sugerencia:** combinaciones adecuadas de `transpose`, `stack`, `reshape` y, ocasionalmente, `flatten` y `diagonal` suele ser más que suficiente. Se recomienda **fuertemente* explorar la dimensionalidad de cada elemento antes de implementar las clases.

## Tensorización

En esta sección nos vamos a ocupar de hacer que el modelo sea más rápido para generar predicciones, observando que incurre en un doble `for` dado que predice en forma individual un escalar para cada observación, para cada clase. Paralelizar ambos vía tensorización suena como una gran vía de mejora de tiempos.

### 1) Diferencias entre `QDA`y `TensorizedQDA`

1. ¿Sobre qué paraleliza `TensorizedQDA`? ¿Sobre las $k$ clases, las $n$ observaciones a predecir, o ambas?

Paraleliza sobre las $k$ clases. 

En el modelo `QDA` se crean $k$ listas de $p \times p$ valores sobre los que luego hay que iterar uno por uno. En cambio `TensorizedQDA` usa un tensor de dimensión $k \times n \times n$ que permite hacer cálculos directamente, paralelizando los cálculos y evitando hacer las $k$ iteraciones hechas en el método `_predict_one`.

2. Analizar los shapes de `tensor_inv_covs` y `tensor_means` y explicar paso a paso cómo es que `TensorizedQDA` llega a predecir lo mismo que `QDA`.

Como se introdujo antes, `TensorizedQDA` usa un tensor de dimensión $k \times p \times p$ para la inversa de sus covarianzas, que tiene tamaño $p \times p$; y de $k \times p \times 1$ para sus medias. 

El cálculo hecho en los métodos `_predict_one` es exactamente el mismo, pero en la clase `QDA` se debe calcular el logaritmo de la probabilidad condicional (hecho en el método individualmente para cada clase `_predict_log_conditionals`) mientras que el modelo `TensorizedQDA` puede hacerlo con las capacidades de cálculo tensorial de un array de numpy de dimensión 3 y calcular el logaritmo de todas las clases a la vez.

Se ve que las predicciones se realizan para cada punto de $x$ que se tiene, pero a la forma de los tensores de medias y covarianzas es se les agrega una dimensión adicional, que corresponde a la categoría que se esta analizando.

Esto llega a predecir lo mismo que QDA debido a que `numpy` utiliza una regla de multiplicación de matrices conocido como *broadcasting* donde en este caso está utilizando la primer dimensión correspondiente a las categorías como dimensión principal, para después propagar las operaciones matriciales en las dimensiones restantes de la matriz. Es como si se hiciera $k$ veces la misma operación de matriz, para cada una de las diferentes categorías.

### 2) Optimización

Debido a la forma cuadrática de QDA, no se puede predecir para $n$ observaciones en una sola pasada (utilizar $X \in \mathbb{R}^{p \times n}$ en vez de $x \in \mathbb{R}^p$) sin pasar por una matriz de $n \times n$ en donde se computan todas las interacciones entre observaciones. Se puede acceder al resultado recuperando sólo la diagonal de dicha matriz, pero resulta ineficiente en tiempo y (especialmente) en memoria. Aún así, es *posible* que el modelo funcione más rápido.

3. Implementar el modelo `FasterQDA` (se recomienda heredarlo de `TensorizedQDA`) de manera de eliminar el ciclo for en el método predict.

La clave en este punto es comprender que la expresión a la que llegamos para la predicción y actualización de la verosimilitud:

$$ (x_s-\mu_c)^T \Sigma_c^{-1} (x_s-\mu_c) $$

Permite determinar la similitud entre dos valores aleatorios de igual distribución de probabilidad conociendo su matriz de covarianza. Esto permite analizar los valores que minimizan ese cálculo (usando para ello la log-verosimilitud desarrollado al inicio del notebook). Esta distancia es llamada **distancia de Mahalanobis**, y su desarrollo usando operaciones tensoriales se puede lograr simplemente multiplicando tensores o fijando algunas variables y haciendo el producto interno de los vectores no fijos en lo que se conoce como una "suma de Einstein": 

In [19]:
class FasterQDA(TensorizedQDA):

    def predict(self, X):
        self.n_samples = X.shape[1]
        self.n_classes = len(self.log_a_priori)

        # Make room for the n_classes dimension on the tensor (1, n_features, n_samples)
        X_expanded = X[np.newaxis, :, :]

        # Repeat all mean matrix for every sample into a single tensor (n_classes, n_features, n_samples)
        self.means_expanded = self.tensor_means @ np.ones((1, self.n_samples))

        # log_a_priori: (n_classes,) -> broadcast to (n_classes, n_samples)
        total_log_posterior = self.log_a_priori[:, np.newaxis] + self._predict_log_conditionals(X_expanded)

        # Pick class with maximum log posterior for each sample
        y_hat = np.argmax(total_log_posterior, axis=0)

        return y_hat.reshape(1, -1)
    
    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.means_expanded

        # Transpose features & samples to multiply tensors (n_classes, n_samples, n_features)
        unbiased_x_t = unbiased_x.transpose(0, 2, 1)

        # Use matmul to solve the Mahalanobis expression: (x_s-mu_c).T*Sigma_c^{-1}*(x_s-mu_c)
        # Sigma is (n_classes, n_features, n_features)
        temp = np.matmul(unbiased_x_t, self.tensor_inv_cov)  # (n_classes, n_samples, n_features)
        prod = np.matmul(temp, unbiased_x)  # (n_classes, n_samples, n_samples)

        # Diagonalize without using for loops
        diag_inner_prod = np.sum(np.multiply(prod, np.eye(self.n_samples)[np.newaxis, :, :]), axis=2)

        log_det = np.log(np.linalg.det(self.tensor_inv_cov))
        return 0.5*log_det[:, np.newaxis] - 0.5 * diag_inner_prod
    
class FasterQDA_Einstein(TensorizedQDA):

    def predict(self, X):
        n_samples = X.shape[1]

        # Make room for the n_classes dimension on the tensor (1, n_features, n_samples)
        X_expanded = X[np.newaxis, :, :]

        # Repeat all mean matrix for every sample into a single tensor (n_classes, n_features, n_samples)
        self.means_expanded = self.tensor_means @ np.ones((1, n_samples))

        # log_a_priori: (n_classes,) -> broadcast to (n_classes, n_samples)
        total_log_posterior = self.log_a_priori[:, np.newaxis] + self._predict_log_conditionals(X_expanded)

        # Pick class with maximum log posterior for each sample
        y_hat = np.argmax(total_log_posterior, axis=0)

        return y_hat.reshape(1, -1)
    
    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.means_expanded

        # Use einsum to multiply tensors. (x_s-mu_c).T*Sigma_c^{-1}*(x_s-mu_c). This expression is already diagonalized
        # Sigma is (n_classes, n_features, n_features)
        inner_prod = np.einsum('kpn,kpq,kqn->kn', unbiased_x, self.tensor_inv_cov, unbiased_x)

        log_det = np.log(np.linalg.det(self.tensor_inv_cov))
        return 0.5*log_det[:, np.newaxis] - 0.5 * inner_prod

4. Mostrar dónde aparece la mencionada matriz de $n \times n$, donde $n$ es la cantidad de observaciones a predecir.

Como se mencionó antes, la expresión de Mahalanois $(x_s-\mu_c)^T \Sigma_c^{-1} (x_s-\mu_c)$ resulta en una matriz auxiliar de $n \times n$ que no es utilizada en su totalidad cuando se diagonaliza. Por este motivo se propone el modelo alternativo `FasterQDA_Einstein` que usa sumas de Einstein para calcular los términos de la diagonal sin crear la matriz completa, con la esperanza de acelerar los cálculos.

5. Demostrar que
$$
diag(A \cdot B) = \sum_{cols} A \odot B^T = np.sum(A \odot B^T, axis=1)
$$ 

es decir, que se puede "esquivar" la matriz de $n \times n$ usando matrices de $n \times p$. También se puede usar, de forma equivalente,

$$
np.sum(A^T \odot B, axis=0).T
$$

queda a preferencia del alumno cuál usar.

Por definición, un elemento de una matriz se resuelve como:

$$ C_{ij} = \sum_{k=1}^{n} A_{ik} \cdot B_{kj} $$

Particularmente, la diagonal de la matriz es el elemento donde $i=j$, de modo que:

$$ \mathrm{diag}(AB)_i = C_{ii} = \sum_{k=1}^{n} A_{ik} \cdot B_{ki} = \sum_{k=1}^{n} A_{ik} \cdot B^T_{ik} = \sum_{k=1}^{n} A_i \odot B_i^T $$

Generalizando para la diagonal entera en lugar de solo un elemento de ella:

$$ \mathrm{diag}(AB) = \sum_{k=1}^{n} A \odot B^T $$

Empíricamente:

In [20]:
n, p = 100, 20
A = np.random.randn(n, p)
B = np.random.randn(p, n)

left = np.diag(A @ B)
right = np.sum(A * B.T, axis=1)

print("Igualdad en ambos lados de la ecuación: ", np.allclose(left, right))

Igualdad en ambos lados de la ecuación:  True


6. Utilizar la propiedad antes demostrada para reimplementar la predicción del modelo `FasterQDA` de forma eficiente en un nuevo modelo `EfficientQDA`.

In [21]:
class EfficientQDA(TensorizedQDA):

    def predict(self, X):
        n_samples = X.shape[1]

        # Make room for the n_classes dimension on the tensor (1, n_features, n_samples)
        X_expanded = X[np.newaxis, :, :]

        # Repeat all mean matrix for every sample into a single tensor (n_classes, n_features, n_samples)
        self.means_expanded = self.tensor_means @ np.ones((1, n_samples))

        # log_a_priori: (n_classes,) -> broadcast to (n_classes, n_samples)
        total_log_posterior = self.log_a_priori[:, np.newaxis] + self._predict_log_conditionals(X_expanded)

        # Pick class with maximum log posterior for each sample
        y_hat = np.argmax(total_log_posterior, axis=0)

        return y_hat.reshape(1, -1)
    
    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.means_expanded

        # Use einsum to multiply tensors. (x_s-mu_c).T*Sigma_c^{-1}
        # Sigma is (n_classes, n_features, n_features)
        A = np.einsum("kpn,kpq->kqn", unbiased_x, self.tensor_inv_cov)

        # Diagonalize the expression using the inner product property to avoid creating a nxn matrix
        inner_prod = np.sum(A * unbiased_x, axis=1)

        log_det = np.log(np.linalg.det(self.tensor_inv_cov))
        return 0.5*log_det[:, np.newaxis] - 0.5 * inner_prod

7. Comparar la performance de las 4 variantes de QDA implementadas hasta ahora (no Cholesky) ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

In [22]:
b = Benchmark(
    X_full, y_full_encoded,
    n_runs = 10000,
    warmup = 20,
    mem_runs = 20,
    test_sz = 0.3,
    same_splits = False
)

to_bench = [QDA, TensorizedQDA, FasterQDA, FasterQDA_Einstein, EfficientQDA]

for model in to_bench:
    b.bench(model)

summ = b.summary(baseline='QDA')
summ[[
    'train_mean_ms', 'test_mean_ms','mean_accuracy',
    'train_speedup', 'test_speedup',
    'train_mem_reduction', 'test_mem_reduction'
]]

Benching params:
Total runs: 10040
Warmup runs: 20
Peak Memory usage runs: 20
Running time runs: 10000
Train size rows (approx): 125
Test size rows (approx): 53
Test size fraction: 0.3


EfficientQDA (TIME): 100%|██████████| 10000/10000 [00:03<00:00, 3256.57it/s]


,train_mean_ms,test_mean_ms,mean_accuracy,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,
QDA,0.142851,1.389113,0.983828,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,0.148373,0.620362,0.983635,0.962779,2.239197,1.016405,0.642755
FasterQDA,0.136503,0.048445,0.983680,1.046504,28.674314,1.033720,0.028011
FasterQDA_Einstein,0.137026,0.049832,0.983781,1.042513,27.875808,1.028427,0.026414
EfficientQDA,0.136895,0.056599,0.983863,1.043510,24.543112,1.032311,0.031544


Se observa que el modelo `FasterQDA` permite entrenar el modelo ligeramente más rápido (posiblemente por efectos computacionales, ya que el entrenamiento no cambia), pero se destaca que tiene una respuesta muchísimo más rápida a costa de mucha mayor utilización de memoria. 

En el modelo `FasterQDA_Einstein` propuesto se mejora aún más la velocidad de respuesta utilizando algo más de memoria y mejorando ligeramente la certeza con respecto al modelo evaluado en el párrafo anterior.

Por último, `EfficientQDA` reduce el tiempo de respuesta con respecto a los modelos anteriores manteniendo similar su certeza, pero conserva ligeramente más la memoria utilizada para ello.

En líneas generales el experimento entrega resultados muy satisfactorios, acelerando muchísimo la respuesta del modelo si se dispone de los recursos de hardware necesarios y sin empeorar la calidad de la misma.

## Cholesky

Hasta ahora todos los esfuerzos fueron enfocados en realizar una predicción más rápida. Los tiempos de entrenamiento (teóricos al menos) siguen siendo los mismos o hasta (minúsculamente) peores, dado que todas las mejoras siguen llamando al método `_fit_params` original de `QDA`.

La descomposición/factorización de [Cholesky](https://en.wikipedia.org/wiki/Cholesky_decomposition#Statement) permite factorizar una matriz definida positiva $A = LL^T$ donde $L$ es una matriz triangular inferior. En particular, si bien se asume que $p \ll n$, invertir la matriz de covarianzas $\Sigma$ para cada clase impone un cuello de botella que podría alivianarse. Teniendo en cuenta que las matrices de covarianza son simétricas y salvo degeneración, definidas positivas, Cholesky como mínimo debería permitir invertir la matriz más rápido.

*Nota: observar que calcular* $A^{-1}b$ *equivale a resolver el sistema* $Ax=b$.

### 3) Diferencias entre implementaciones de `QDA_Chol`

8. Si una matriz $A$ tiene fact. de Cholesky $A=LL^T$, expresar $A^{-1}$ en términos de $L$. ¿Cómo podría esto ser útil en la forma cuadrática de QDA?

$$ A=LL^T \Rightarrow A^{-1} = (LL^T)^{-1} = (L^T)^{-1} L^{-1} $$

Dado que la matriz de covarianzas $\Sigma$ es **siempre simétrica**, es muy fácil descomponerla en términos de una matriz triangular y por lo tanto puede aplicarse la descomposición de Cholesky sin problema. Esto permite que, al calcular la distancia de Mahalanobis, si consideramos $z=(x_s-\mu_c)$:

$$ z^T \Sigma_c^{-1} z \Rightarrow z^T (L^T)^{-1} L^{-1} z $$

Si ahora se resuelve una parte de la ecuación considerando $y=L^{-1} z$, se puede también considerar $y^T=z^T (L^T)^{-1} $; de modo que el cálculo de la distancia de Mahalanois se puede expresar ahora como:

$$ y^T y $$

De modo que es posible resolver únicamente $ y = L^{-1} (x_s-\mu_c) $ y trasponer el resultado para obtener una expresión equivalente que acelera mucho los cálculos.

9.  Explicar las diferencias entre `QDA_Chol1` y `QDA` y cómo `QDA_Chol1` llega, paso a paso, hasta las predicciones.

La diferencia principal es el uso de la factorización de Cholesky que se describió en el punto anterior en lugar del cálculo directo de la inversa de la matriz de covarianzas.

Para el cálculo del logaritmo condicional previene el uso de funciones inversas y productos complejos entre matrices, acelerando los cálculos y evitando inestabilidades por las funciones matemáticas usadas. Además por propiedades de las matrices triangulares, la operación matricial para el cálculo del logaritmo de verosimilitud puede reemplazarse por la **sumatoria del cuadrado de $y$**, y la operacion de determinante puede reemplazarse por el **producto de los elementos de la diagonal**.
    
10. ¿Cuáles son las diferencias entre `QDA_Chol1`, `QDA_Chol2` y `QDA_Chol3`?

Difieren en que guardan el valor de L en lugar de su inversa (`QDA_Chol2`) o cómo se calcula la inversa de manera eficiente usando `scipy.linalg.lapack.dtrtri` (`QDA_Chol3`). En la predicción, la única diferencia es con `QDA_Chol2`, que usa el método `scipy.linalg.solve_triangular` para calcular el logaritmo de verosimilitud de manera más eficiente.
    
11. Comparar la performance de las 7 variantes de QDA implementadas hasta ahora ¿Qué se observa?¿Hay alguna de las implementaciones de `QDA_Chol` que sea claramente mejor que las demás? ¿Alguna que sea peor?

In [23]:
to_bench = [QDA_Chol1, QDA_Chol2, QDA_Chol3]

for model in to_bench:
    b.bench(model)

summ = b.summary(baseline='QDA')
summ[[
    'train_mean_ms', 'test_mean_ms','mean_accuracy',
    'train_speedup', 'test_speedup',
    'train_mem_reduction', 'test_mem_reduction'
]]

QDA_Chol3 (TIME): 100%|██████████| 10000/10000 [00:10<00:00, 924.33it/s]


,train_mean_ms,test_mean_ms,mean_accuracy,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,
QDA,0.142851,1.389113,0.983828,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,0.148373,0.620362,0.983635,0.962779,2.239197,1.016405,0.642755
FasterQDA,0.136503,0.048445,0.983680,1.046504,28.674314,1.033720,0.028011
FasterQDA_Einstein,0.137026,0.049832,0.983781,1.042513,27.875808,1.028427,0.026414
EfficientQDA,0.136895,0.056599,0.983863,1.043510,24.543112,1.032311,0.031544
QDA_Chol1,0.161412,0.845809,0.983670,0.885008,1.642349,1.020523,1.005820
QDA_Chol2,0.120216,1.720458,0.983498,1.188281,0.807409,1.031943,1.014498
QDA_Chol3,0.123032,0.840302,0.983694,1.161084,1.653112,1.043376,1.025312


Todos los métodos se mantienen similares en cuanto a certeza, indicando que los cálculos realizados por el momento son todos equivalentes. El modelo `QDA_Chol1` empeora los tiempos de entrenamiento respecto a `QDA`, pero los modelos sucesores `QDA_Chol2` y `QDA_Chol3` obtienen aceleraciones significativas. En su uso con los datasets de validación si bien se observan mejoras respecto al `QDA` original no alcanza las elevadas velocidades de los modelos evaluados previamente, pero sí conservan mejor la memoria utilizada. El modelo `QDA_Chol3` es el que mejor performa bajo estos parámetros.

### 4) Optimización

12. Implementar el modelo `TensorizedChol` paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de `QDA_Chol`, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.

In [29]:
class TensorizedChol(QDA_Chol3):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X,y)

        # stack onto new dimension
        self.tensor_L_invs = np.stack(self.L_invs)
        self.tensor_means = np.stack(self.means)

    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.tensor_means
        y = np.matmul(self.tensor_L_invs, unbiased_x)

        return -np.log(np.diagonal(self.tensor_L_invs, axis1=1, axis2=2)).sum(axis=1) - 0.5 * (y**2).sum(axis=1).squeeze(-1)

    def _predict_one(self, x):
        # return the class that has maximum a posteriori probability
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))

13. Implementar el modelo `EfficientChol` combinando los insights de `EfficientQDA` y `TensorizedChol`. Si se desea, se puede implementar `FasterChol` como ayuda, pero no se contempla para el punto.

In [25]:
class EfficientChol(TensorizedChol):

    def predict(self, X):
        n_samples = X.shape[1]

        # Make room for the n_classes dimension on the tensor (1, n_features, n_samples)
        X_expanded = X[np.newaxis, :, :]

        # Repeat all mean matrix for every sample into a single tensor (n_classes, n_features, n_samples)
        self.means_expanded = self.tensor_means @ np.ones((1, n_samples))

        # log_a_priori: (n_classes,) -> broadcast to (n_classes, n_samples)
        total_log_posterior = self.log_a_priori[:, np.newaxis] + self._predict_log_conditionals(X_expanded)

        # Pick class with maximum log posterior for each sample
        y_hat = np.argmax(total_log_posterior, axis=0)

        return y_hat.reshape(1, -1)
    
    def _predict_log_conditionals(self,x):
        unbiased_x = x - self.means_expanded
        y = np.einsum('cij,cjk->cik', self.tensor_L_invs, unbiased_x)

        log_det = -np.log(np.diagonal(self.tensor_L_invs, axis1=1, axis2=2)).sum(axis=1)
        log_det = log_det[:, np.newaxis]
        return log_det - 0.5 * (y**2).sum(axis=1)

14. Comparar la performance de las 9 variantes de QDA implementadas ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

In [30]:
to_bench = [TensorizedChol, EfficientChol]

for model in to_bench:
    b.bench(model)

summ = b.summary(baseline='QDA')
summ[[
    'train_mean_ms', 'test_mean_ms','mean_accuracy',
    'train_speedup', 'test_speedup',
    'train_mem_reduction', 'test_mem_reduction'
]]

EfficientChol (TIME): 100%|██████████| 10000/10000 [00:02<00:00, 3517.53it/s]


,train_mean_ms,test_mean_ms,mean_accuracy,train_speedup,test_speedup,train_mem_reduction,test_mem_reduction
model,,,,,,,
QDA,0.142851,1.389113,0.983828,1.000000,1.000000,1.000000,1.000000
TensorizedQDA,0.148373,0.620362,0.983635,0.962779,2.239197,1.016405,0.642755
FasterQDA,0.136503,0.048445,0.983680,1.046504,28.674314,1.033720,0.028011
FasterQDA_Einstein,0.137026,0.049832,0.983781,1.042513,27.875808,1.028427,0.026414
EfficientQDA,0.136895,0.056599,0.983863,1.043510,24.543112,1.032311,0.031544
QDA_Chol1,0.161412,0.845809,0.983670,0.885008,1.642349,1.020523,1.005820
QDA_Chol2,0.120216,1.720458,0.983498,1.188281,0.807409,1.031943,1.014498
QDA_Chol3,0.123032,0.840302,0.983694,1.161084,1.653112,1.043376,1.025312
TensorizedChol,0.133383,0.495730,0.938281,1.070986,2.802155,1.038708,0.608855


Se observa que los nuevos modelos combinan la rapidez de los modelos individuales vistos anteriormente, acelerando el tiempo de ambos conjuntos, entrenamiento y evaluación, sin ningún coste adicional en memoria. Sin embargo sí se observa una reducción en la certeza que puede resultar muy perjudicial según la aplicación a la que se destine el modelo.